### Colab

In [1]:
# from google.colab import drive
# drive.mount('drive', force_remount=True)

Mounted at drive


### We evaluate using seqeval

In [2]:
# !pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=5c914f4f502552cf9b9649946817618b19b66838caf2f44765b624429031c4e5
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


### Imports

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from tqdm import tqdm
from seqeval.metrics import classification_report

device = 'cuda' if torch.cuda.is_available() else 'cpu'

### We need NERDataset to load the saved DataLoaders

In [4]:
class NERDataset(Dataset):
    def __init__(self, X, Y, vocab2id, label2id, max_seq_length):
        self.X, self.Y = X, Y
        self.max_seq_length = max_seq_length
        self.vocab2id = vocab2id
        self.label2id = label2id

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        tokens, labels = self.X[idx], self.Y[idx]

        if len(tokens) > self.max_seq_length:
            tokens = tokens[: self.max_seq_length]
            labels = labels[: self.max_seq_length]
        else:
            tokens = tokens + ['<PAD>'] * (self.max_seq_length - len(tokens))
            labels = labels + ['O'] * (self.max_seq_length - len(labels))

        tokens_tensor = torch.tensor(
            [self.vocab2id.get(token, self.vocab2id['<UNK>']) for token in tokens]).to(device)
        labels_tensor = torch.tensor([self.label2id.get(label) for label in labels]).to(device)
        return tokens_tensor, labels_tensor

### Load Saved Loaders

In [9]:
loaders = torch.load('data/loaders.pt', weights_only=False, map_location=device)

train_loader = loaders['train']
val_loader = loaders['val']
test_loader = loaders['test']

vocab2id = loaders['vocab2id']
id2vocab = loaders['id2vocab']

label2id = loaders['label2id']
id2label = loaders['id2label']

### To load saved LSTM models

In [5]:
class MyLSTM(nn.Module):
    def __init__(self, input_size, emb_size, hidden_size, output_size, bidirectional, num_layers):
        super(MyLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_directions = 2 if bidirectional else 1
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, emb_size)
        self.lstm = nn.LSTM(emb_size, hidden_size, bidirectional=bidirectional, num_layers=num_layers, batch_first=True)
        self.clf1 = nn.Linear(hidden_size * self.num_directions, output_size)

    def forward(self, X):
        e = self.embedding(X)
        h0 = torch.zeros(self.num_directions * self.num_layers, X.shape[0], self.hidden_size).to(device)
        c0 = torch.zeros(self.num_directions * self.num_layers, X.shape[0], self.hidden_size).to(device)
        o, (h0,c0) = self.lstm(e, (h0,c0))

        return self.clf1(o)

### To load saved GRU models

In [6]:
class MyGRU(nn.Module):
    def __init__(self, input_size, emb_size, hidden_size, output_size, bidirectional, num_layers):
        super(MyGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_directions = 2 if bidirectional else 1
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, bidirectional=bidirectional, num_layers=num_layers, batch_first=True)
        self.clf1 = nn.Linear(hidden_size * self.num_directions, output_size)

    def forward(self, X):
        e = self.embedding(X)
        h0 = torch.zeros(self.num_directions * self.num_layers, X.shape[0], self.hidden_size).to(device)
        o, h0 = self.gru(e, h0)
        return self.clf1(o)

### Inference

In [10]:
import os

path = 'models/'
available_models = os.listdir(path)

for modelname in available_models:
    modelpath = f'{path}/{modelname}'

    loaded = torch.load(modelpath, weights_only=False, map_location=device)

    if 'lstm' in modelname:
      class_type = MyLSTM
    else:
      class_type = MyGRU

    model_inference = class_type(input_size=loaded['input_size'], emb_size=loaded['emb_size'], hidden_size=loaded['hidden_size'], output_size=loaded['output_size'],
              bidirectional=loaded['bidirectional'], num_layers=loaded['num_layers']).to(device)

    print(model_inference.load_state_dict(loaded['state_dict']))


    Y_pred, Y_test = [],[]
    with torch.no_grad():
        for Xs, Ys in tqdm(test_loader):
            Xs, Ys = Xs.to(device), Ys.to(device)
            pred_y = model_inference.forward(Xs)

            pred_y = torch.argmax(model_inference.forward(Xs).view(-1, pred_y.shape[-1]), dim=-1)
            Ys = Ys.view(-1)

            Y_pred.append([id2label[_id_.item()] for _id_ in pred_y])

            Y_test.append([id2label[_id_.item()] for _id_ in Ys])

    print(f'model: {loaded["model_name"]}\n{classification_report(Y_test, Y_pred)}\n')


<All keys matched successfully>


100%|██████████| 17/17 [00:11<00:00,  1.45it/s]


model: gru-1-unidirectional-10-epochs
                       precision    recall  f1-score   support

                  DDF       0.60      0.62      0.61       234
  anatomical%location       0.61      0.50      0.55        22
               animal       0.41      0.37      0.39        19
             bacteria       0.62      0.50      0.55        84
 biomedical%technique       0.50      0.28      0.36        32
             chemical       0.56      0.44      0.49        52
   dietary%supplement       0.64      0.50      0.56        42
                 drug       0.80      0.50      0.62         8
                 food       0.00      0.00      0.00         4
                 gene       0.00      0.00      0.00         9
                human       0.55      0.60      0.58        85
           microbiome       0.47      0.58      0.52        74
statistical%technique       0.67      0.40      0.50         5

            micro avg       0.57      0.53      0.55       670
            mac

100%|██████████| 17/17 [00:55<00:00,  3.28s/it]


model: gru-2-bidirectional-10-epochs
                       precision    recall  f1-score   support

                  DDF       0.76      0.74      0.75       234
  anatomical%location       0.84      0.73      0.78        22
               animal       0.75      0.47      0.58        19
             bacteria       0.69      0.60      0.64        84
 biomedical%technique       0.43      0.28      0.34        32
             chemical       0.52      0.56      0.54        52
   dietary%supplement       0.84      0.64      0.73        42
                 drug       1.00      0.38      0.55         8
                 food       0.00      0.00      0.00         4
                 gene       0.50      0.11      0.18         9
                human       0.73      0.72      0.73        85
           microbiome       0.74      0.86      0.80        74
statistical%technique       0.43      0.60      0.50         5

            micro avg       0.72      0.67      0.69       670
            macr

100%|██████████| 17/17 [00:13<00:00,  1.29it/s]
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


model: lstm-1-unidirectional-10-epochs
                       precision    recall  f1-score   support

                  DDF       0.62      0.63      0.62       234
  anatomical%location       0.71      0.55      0.62        22
               animal       0.23      0.16      0.19        19
             bacteria       0.67      0.60      0.63        84
 biomedical%technique       0.35      0.19      0.24        32
             chemical       0.55      0.40      0.47        52
   dietary%supplement       0.63      0.45      0.53        42
                 drug       0.80      0.50      0.62         8
                 food       0.00      0.00      0.00         4
                 gene       0.00      0.00      0.00         9
                human       0.50      0.55      0.53        85
           microbiome       0.46      0.68      0.55        74
statistical%technique       0.25      0.20      0.22         5

            micro avg       0.56      0.54      0.55       670
            ma

100%|██████████| 17/17 [06:01<00:00, 21.24s/it]


model: lstm-2-bidirectional-10-epochs
                       precision    recall  f1-score   support

                  DDF       0.72      0.77      0.74       234
  anatomical%location       0.67      0.73      0.70        22
               animal       0.65      0.58      0.61        19
             bacteria       0.71      0.71      0.71        84
 biomedical%technique       0.50      0.38      0.43        32
             chemical       0.57      0.44      0.50        52
   dietary%supplement       0.63      0.64      0.64        42
                 drug       0.83      0.62      0.71         8
                 food       0.00      0.00      0.00         4
                 gene       0.12      0.11      0.12         9
                human       0.74      0.75      0.74        85
           microbiome       0.74      0.86      0.80        74
statistical%technique       0.33      0.40      0.36         5

            micro avg       0.69      0.69      0.69       670
            mac